# Turner Lab M1 MPTP Dataset - NWB Usage Guide

**Dataset Overview:**
This dataset contains single-unit electrophysiology recordings from primary motor cortex (M1) of parkinsonian macaque monkeys performing flexion/extension motor tasks. The data investigates motor encoding deficits in MPTP-induced parkinsonism, comparing pyramidal tract neurons (PTNs) versus corticostriatal neurons (CSNs).

**Key Features:**
- **Single-unit recordings**: Spike times and waveforms from M1 neurons
- **Motor behavior**: Flexion/extension task with analog kinematics
- **Cell type identification**: Antidromic stimulation to classify PTNs vs CSNs
- **Disease model**: MPTP-treated parkinsonian macaque monkeys
- **Electrode mapping**: Systematic cortical penetrations with stereotactic coordinates

**Data Organization:**
- Each NWB file represents one recording session from one penetration depth
- Session IDs follow pattern: `{subject_id}++{FileName}++{PreMPTP|PostMPTP}++Depth{depth_um}um++{YearMonthDay}`
- Example: `V++{v0502}++PostMPTP++Depth19180um++20000121` indicates monkey V, post-MPTP condition, 19.18mm depth, recorded Jan 21, 2000

## Streaming NWB Files from DANDI 

We recommend using the DANDI Python client to access the NWB files directly from the DANDI archive without downloading them locally.

In [ ]:
import h5py
import remfile
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

# Connect to DANDI and get the dandiset
dandiset_id = "001636"
client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id, "draft")

# Get all assets - users can filter this list to select any session
assets = dandiset.get_assets()
assets_list = list(assets)
print(f"Total assets in dandiset: {len(assets_list)}")

# Filter for NWB files only
nwb_assets = [a for a in assets_list if a.path.endswith(".nwb")]
print(f"NWB files: {len(nwb_assets)}")

# We select this specific session because it is "complete" - it has all available data types:
# - 2 PTNs (pyramidal tract neurons) identified via antidromic stimulation
# - EMG recordings from multiple arm muscles
# - LFP data
# - Perturbation trials (52 out of 80 total trials)
# - Balanced flexion/extension movements (40 each)
#
# The only missing feature is receptive field locations (not recorded for these units).
# See dandiset_session_metadata.csv for a full inventory of all 298 sessions.
asset_path = "sub-V/sub-V_ses-V++v5811++PostMPTP++Depth18300um++20000331_behavior+ecephys.nwb"
asset = next(a for a in nwb_assets if a.path == asset_path)
print(f"Selected asset: {asset.path}")

# Stream the NWB file directly from DANDI (no download required)
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()

Fist, we can visualize the HTML representation of the nwbfile to get an overview of its contents.

In [ ]:
nwbfile

## Understanding the Trial Structure

The behavioral task is a visuomotor step-tracking paradigm where monkeys make rapid elbow flexion/extension movements to capture visual targets. Each trial follows a stereotyped sequence of events that we can examine through the trials table.

In [ ]:
nwbfile.trials

Trials are stored as a table in NWB. We can convert it to a pandas DataFrame to explore the structure of the trial events

In [ ]:
trials_df = nwbfile.trials.to_dataframe()
trials_df.sample(n=5)

### Trial Event Sequence

Let's examine the key columns that define the temporal structure of each trial:

In [ ]:
columns = [
    "start_time",
    "center_target_appearance_time",
    "lateral_target_appearance_time",
    "cursor_departure_time",
    "movement_type",
    "reward_time",
]
trials_df[columns].head(n=5)

Each trial progresses through distinct phases:

1. **Trial start**: Recording begins with a variable baseline period
2. **Center target appearance**: A center target appears on screen, cueing the monkey to align the cursor and hold
3. **Center hold period**: The monkey maintains position for 1-2 seconds (randomized to prevent anticipation)
4. **Lateral target appearance (go cue)**: A peripheral target appears, signaling the monkey to move
5. **Cursor departure**: The monkey initiates movement, exiting the center zone
6. **Movement execution**: Rapid ballistic movement toward the target (flexion or extension)
7. **Reward**: Liquid reward delivered upon successful target acquisition

The time differences between these events reveal reaction times, movement durations, and other behaviorally relevant measures.

To graphically understand the trial structure, we can look at the following visualization that is using the data in the NWB file:

In [ ]:
from trial_structure_plot import plot_trial_structure

plot_trial_structure(trials_df, max_trials=10);

The visualization above shows the temporal structure of individual trials as horizontal stacked bars. Each row represents one trial, with colored segments indicating different task phases:

- **Baseline** (gray): Variable waiting period before the task begins
- **Holding at Center** (teal): The monkey maintains cursor position at the center target, waiting for the go cue
- **Time to React** (yellow): The interval between go cue (lateral target appearance) and cursor departure from the center zone
- **Movement** (coral): Active movement execution toward the peripheral target
- **Holding at Target** (purple): The monkey holds position at the target before reward delivery
- **Post-Reward** (sky blue): Brief period after reward before trial ends

Trials labeled "Aborted" were terminated early (the monkey broke fixation before the go cue). Notice the consistent structure across trials, with the main variability coming from the randomized center hold duration (1-2 seconds) and individual differences in reaction time and movement speed.

### Derived Kinematic Measures

In addition to trial event times, the trials table includes derived kinematic parameters computed from post-hoc analysis of the elbow angle velocity trace. These measures characterize the movement itself:

| Column | Description | Units |
|--------|-------------|-------|
| `derived_movement_onset_time` | Time when movement began (detected from velocity trace) | seconds |
| `derived_movement_end_time` | Time when movement ended (velocity returned to baseline) | seconds |
| `derived_peak_velocity` | Maximum angular velocity during the movement | degrees/second |
| `derived_peak_velocity_time` | Time of peak velocity | seconds |
| `derived_movement_amplitude` | Total angular displacement (end position - start position) | degrees |
| `derived_end_position` | Final elbow angle at movement end | degrees |

The "derived" prefix indicates these values were computed offline from kinematic analysis, not recorded directly during the experiment. The detection algorithm used position, velocity, and duration criteria (exact parameters not documented in source data).

**Sign conventions:**
- Positive velocities and amplitudes indicate extension movements
- Negative velocities and amplitudes indicate flexion movements

In [ ]:
# Access derived kinematic measures
kinematic_columns = [
    'derived_movement_onset_time',
    'derived_movement_end_time',
    'derived_peak_velocity',
    'derived_peak_velocity_time',
    'derived_movement_amplitude',
    'derived_end_position',
    'movement_type'
]
trials_df[kinematic_columns].head(n=5)

In [ ]:
# Compute commonly used derived measures
# Note: Some trials may have NaN values if kinematic detection failed

# Movement duration (from movement onset to movement end)
trials_df['movement_duration'] = (
    trials_df['derived_movement_end_time'] - trials_df['derived_movement_onset_time']
)

# Reaction time (from go cue to cursor departure)
trials_df['reaction_time'] = (
    trials_df['cursor_departure_time'] - trials_df['lateral_target_appearance_time']
)

# Filter to complete trials only
complete_trials = trials_df.dropna(subset=['derived_movement_end_time'])

print(f"Complete trials with kinematic data: {len(complete_trials)} / {len(trials_df)}")
print(f"\nMovement duration: {complete_trials['movement_duration'].mean()*1000:.0f} +/- {complete_trials['movement_duration'].std()*1000:.0f} ms")
print(f"Reaction time: {complete_trials['reaction_time'].mean()*1000:.0f} +/- {complete_trials['reaction_time'].std()*1000:.0f} ms")
print(f"Peak velocity: {complete_trials['derived_peak_velocity'].abs().mean():.1f} +/- {complete_trials['derived_peak_velocity'].abs().std():.1f} deg/s")

### Torque Perturbation Trials

A subset of trials included brief mechanical perturbations delivered via the torque motor during the center hold period. These perturbations caused rapid muscle stretch, allowing researchers to study proprioceptive responses in M1 neurons (see Pasquereau & Turner, 2013).

**Note:** Not all sessions include perturbation trials. Sessions without perturbations will have `torque_perturbation_type = "none"` for all trials and `torque_perturbation_onset_time = NaN`.

| Column | Description | Values |
|--------|-------------|--------|
| `torque_perturbation_type` | Direction of the perturbation | `"flexion"`, `"extension"`, or `"none"` |
| `torque_perturbation_onset_time` | Time when perturbation was delivered | seconds (NaN if no perturbation) |

**Perturbation characteristics (when present):**
- Delivered 150-500 ms after center target appearance (during hold period)
- Brief torque pulse causing ~5-10 degree displacement
- Direction (flexion/extension) was independent of the upcoming movement direction
- Present in approximately 65% of trials in sessions that included perturbations

**Scientific purpose:** These trials enabled analysis of how M1 neurons respond to passive muscle stretch, separate from active movement. The muscle stretch study found that PTNs showed degraded directional selectivity to perturbations in the parkinsonian state.

In [ ]:
# Access torque perturbation data
perturbation_columns = ['torque_perturbation_type', 'torque_perturbation_onset_time', 'movement_type']
trials_df[perturbation_columns].head(n=10)

In [ ]:
# Analyze perturbation trial distribution
# Note: Not all sessions include perturbation trials. Sessions without perturbations
# will have torque_perturbation_type = "none" for all trials.

perturbation_counts = trials_df['torque_perturbation_type'].value_counts()
n_perturbation_trials = len(trials_df[trials_df['torque_perturbation_type'] != 'none'])

# Check if this session has perturbation data
if n_perturbation_trials == 0:
    print("This session does not include torque perturbation trials.")
    print("All trials have torque_perturbation_type = 'none'")
else:
    print(f"This session includes perturbation trials ({n_perturbation_trials} / {len(trials_df)} trials)\n")
    print("Perturbation trial counts:")
    for ptype, count in perturbation_counts.items():
        print(f"  {ptype}: {count} trials ({100*count/len(trials_df):.1f}%)")
    
    # Cross-tabulate perturbation direction vs movement direction
    print("\nPerturbation direction vs Movement direction:")
    print(trials_df.groupby(['torque_perturbation_type', 'movement_type']).size().unstack(fill_value=0))

### Isolation Monitoring Stimulation

In some recording sessions, single antidromic stimulation pulses were delivered early in certain trials to activate otherwise silent neurons and monitor unit isolation. This was particularly important for intratelencephalic-type (IT) cortical neurons (including corticostriatal neurons), which have very low spontaneous firing rates.

**Background:** Some neurons rarely spike spontaneously but can be reliably activated by antidromic stimulation from their projection target (striatum, thalamus, or cerebral peduncle). Delivering occasional stimulation pulses during recording confirmed that the unit remained well-isolated throughout the session.

| Column | Description | Values |
|--------|-------------|--------|
| `isolation_monitoring_stim_time` | Time when stimulation pulse was delivered | seconds (NaN if no stimulation in that trial) |
| `isolation_monitoring_stim_site` | Anatomical target of the stimulation | `"Str"` (striatum), `"Thal"` (thalamus), `"Ped"` (peduncle), or empty string |

**Characteristics:**
- Stimulation pulses delivered ~244-245 ms after trial start (early in the baseline period)
- Present in approximately 43% of sessions (171 out of 400 session files)
- Most commonly targeting thalamus (117 sessions), followed by striatum (51 sessions)
- When present, stimulation typically delivered in a subset of trials within the session

**Note:** If you observe spike times that appear suspiciously constant across trials (~245-256 ms after trial start), these likely reflect antidromically-evoked spikes from isolation monitoring, not spontaneous activity.

In [ ]:
# Access isolation monitoring stimulation data
stim_columns = ['isolation_monitoring_stim_time', 'isolation_monitoring_stim_site', 'start_time']
trials_df[stim_columns].head(n=10)

In [ ]:
# Check if this session has isolation monitoring stimulation
stim_trials = trials_df[trials_df['isolation_monitoring_stim_time'].notna()]
n_stim_trials = len(stim_trials)

if n_stim_trials == 0:
    print("This session does not include isolation monitoring stimulation.")
    print("All trials have isolation_monitoring_stim_time = NaN")
else:
    print(f"This session includes isolation monitoring stimulation ({n_stim_trials} / {len(trials_df)} trials)\n")
    
    # Get stimulation site (should be consistent within session)
    stim_site = stim_trials['isolation_monitoring_stim_site'].iloc[0]
    print(f"Stimulation site: {stim_site}")
    
    # Calculate timing relative to trial start
    stim_trials = stim_trials.copy()
    stim_trials['stim_time_relative'] = (
        stim_trials['isolation_monitoring_stim_time'] - stim_trials['start_time']
    )
    
    mean_stim_time = stim_trials['stim_time_relative'].mean() * 1000  # Convert to ms
    std_stim_time = stim_trials['stim_time_relative'].std() * 1000
    
    print(f"Stimulation timing: {mean_stim_time:.1f} +/- {std_stim_time:.1f} ms after trial start")

## Accessing Neurons/Units Spikes 

In [ ]:
nwbfile.units

In [ ]:
units_df = nwbfile.units.to_dataframe()
units_df

## Acquisition Information

In [ ]:
nwbfile.acquisition

## Electrode Configuration and Recording Setup

The dataset includes both recording and stimulation electrodes with detailed anatomical information.

In [ ]:
# Examine electrode configuration
from pynwb import read_nwb

electrodes_df = nwbfile.electrodes.to_dataframe()
print("Electrode Configuration:")
print(f"Total electrodes: {len(electrodes_df)}")
print(f"Recording electrodes: {len(electrodes_df[~electrodes_df['is_stimulation']])}")
print(f"Stimulation electrodes: {len(electrodes_df[electrodes_df['is_stimulation']])}")

print("\nRecording electrode details:")
recording_electrode = electrodes_df[~electrodes_df['is_stimulation']].iloc[0]
print(f"  Chamber coordinates: A/P={recording_electrode['chamber_grid_ap_mm']:.2f}mm, M/L={recording_electrode['chamber_grid_ml_mm']:.2f}mm")
print(f"  Insertion depth: {recording_electrode['chamber_insertion_depth_mm']:.2f}mm")
print(f"  Recording site index: {recording_electrode['recording_site_index']}")
print(f"  Recording session index: {recording_electrode['recording_session_index']}")

print("\nStimulation electrodes:")
stim_electrodes = electrodes_df[electrodes_df['is_stimulation']]
for _, electrode in stim_electrodes.iterrows():
    print(f"  {electrode['location']}: {electrode['stim_notes']}")

# Display electrode table
electrodes_df[['location', 'group_name', 'is_stimulation', 'chamber_grid_ap_mm', 'chamber_grid_ml_mm', 'chamber_insertion_depth_mm']]

In [ ]:
electrodes_df

## Trial Structure and Motor Behavior

First, let's examine the experimental trials which provide the temporal structure for all other analyses.

In [ ]:
# Analyze trial structure
trials_df = nwbfile.trials.to_dataframe()
print(f"Number of trials: {len(trials_df)}")
print("\nTrial metadata columns:")
for col in trials_df.columns:
    print(f"  {col}")

# Display trials table
trials_df

## Single-Unit Activity Analysis

Now let's examine the single-unit spike data and how it relates to the trial structure.

In [ ]:
# Examine units table
units_df = nwbfile.units.to_dataframe()
units_df

## Trialized Spike Analysis

Let's analyze spikes within the context of behavioral trials.

In [ ]:
# Trialized spike analysis
unit_id = 0
spike_times = nwbfile.units['spike_times'][unit_id]

print(f"Unit {unit_id} analysis:")
print(f"  Total spikes: {len(spike_times)}")
print(f"  Recording duration: {spike_times[-1] - spike_times[0]:.2f} seconds")
print(f"  Mean firing rate: {len(spike_times) / (spike_times[-1] - spike_times[0]):.2f} Hz")

# Create trial-aligned spike raster
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Trial-aligned spike raster
trial_spikes = []
for trial_idx, trial in trials_df.iterrows():
    trial_start = trial['start_time']
    trial_stop = trial['stop_time']
    
    # Find spikes within this trial
    trial_spike_times = spike_times[(spike_times >= trial_start) & (spike_times <= trial_stop)]
    
    # Convert to trial-relative times
    relative_spike_times = trial_spike_times - trial_start
    trial_spikes.append(relative_spike_times)
    
    # Plot spikes for this trial
    if len(relative_spike_times) > 0:
        ax1.scatter(relative_spike_times, np.full(len(relative_spike_times), trial_idx), 
                   s=1, color='black', alpha=0.7)

ax1.set_xlabel('Time relative to trial start (s)')
ax1.set_ylabel('Trial number')
ax1.set_title('Trial-aligned spike raster')
ax1.grid(True, alpha=0.3)

# PSTH across trials
# Bin spikes relative to trial start
bin_size = 0.1  # 100ms bins
max_trial_duration = trials_df['stop_time'].max() - trials_df['start_time'].min()
bins = np.arange(0, max_trial_duration + bin_size, bin_size)

# Collect all trial-relative spike times
all_relative_spikes = np.concatenate([spikes for spikes in trial_spikes if len(spikes) > 0])

if len(all_relative_spikes) > 0:
    counts, _ = np.histogram(all_relative_spikes, bins=bins)
    firing_rate = counts / (bin_size * len(trials_df))  # Average across trials
    
    ax2.plot(bins[:-1], firing_rate, linewidth=2)
    ax2.set_xlabel('Time relative to trial start (s)')
    ax2.set_ylabel('Firing rate (Hz)')
    ax2.set_title('Peri-stimulus time histogram (PSTH)')
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\\nTrialized analysis:")
print(f"  Trials with spikes: {sum(1 for spikes in trial_spikes if len(spikes) > 0)}/{len(trials_df)}")
print(f"  Mean spikes per trial: {np.mean([len(spikes) for spikes in trial_spikes]):.2f}")

## Trialized Kinematic Analysis (TimeSeriesElbowVelocity)

Now let's examine how the kinematic data aligns with trials.

In [ ]:
# Trialized kinematic analysis
# Find TimeSeriesElbowVelocity
elbow_velocity_series = nwbfile.acquisition['TimeSeriesElbowVelocity']

print(f"Kinematic data: {elbow_velocity_series.name}")
print(f"  Description: {elbow_velocity_series.description}")
print(f"  Data shape: {elbow_velocity_series.data.shape}")
print(f"  Sampling rate: {elbow_velocity_series.rate} Hz")
print(f"  Duration: {elbow_velocity_series.data.shape[0] / elbow_velocity_series.rate:.2f} seconds")

# Extract trial-aligned kinematic data
kinematic_data = elbow_velocity_series.data[:]
sampling_rate = elbow_velocity_series.rate

# Plot trial-aligned kinematics
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

trial_kinematics = []
for trial_idx, trial in trials_df.iterrows():
    trial_start = trial['start_time']
    trial_stop = trial['stop_time']
    
    # Convert trial times to sample indices
    start_sample = int(trial_start * sampling_rate)
    stop_sample = int(trial_stop * sampling_rate)
    
    # Extract kinematic data for this trial
    trial_data = kinematic_data[start_sample:stop_sample]
    trial_time = np.arange(len(trial_data)) / sampling_rate
    
    trial_kinematics.append(trial_data)
    
    # Plot individual trial (show first 10 trials)
    if trial_idx < 10:
        ax1.plot(trial_time, trial_data, alpha=0.7, linewidth=1, label=f'Trial {trial_idx+1}')

ax1.set_xlabel('Time relative to trial start (s)')
ax1.set_ylabel('Elbow Velocity')
ax1.set_title('Trial-aligned Elbow Velocity (first 10 trials)')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Average across trials
# Find minimum trial length for alignment
min_trial_length = min(len(trial_data) for trial_data in trial_kinematics)

# Truncate all trials to minimum length and average
aligned_trials = np.array([trial_data[:min_trial_length] for trial_data in trial_kinematics])
mean_kinematic = np.mean(aligned_trials, axis=0)
std_kinematic = np.std(aligned_trials, axis=0)

time_axis = np.arange(min_trial_length) / sampling_rate

ax2.plot(time_axis, mean_kinematic, 'b-', linewidth=2, label='Mean')
ax2.fill_between(time_axis, 
                mean_kinematic - std_kinematic, 
                mean_kinematic + std_kinematic, 
                alpha=0.3, color='blue', label='±1 SD')
ax2.set_xlabel('Time relative to trial start (s)')
ax2.set_ylabel('Elbow Velocity')
ax2.set_title('Average trial-aligned Elbow Velocity')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\\nTrialized kinematic analysis:")
print(f"  Number of trials: {len(trial_kinematics)}")
print(f"  Average trial duration: {np.mean([len(trial)/sampling_rate for trial in trial_kinematics]):.2f} seconds")
print(f"  Kinematic range: {np.min(kinematic_data):.2f} to {np.max(kinematic_data):.2f}")

## Waveform Analysis

Examine spike waveform characteristics for unit classification.

In [ ]:
# Analyze spike waveforms
unit_id = 0

# Get waveform data
waveform_mean = nwbfile.units['waveform_mean'][unit_id]
waveform_sd = nwbfile.units['waveform_sd'][unit_id]

# Create time axis (20kHz sampling, 1.6ms window)
sampling_rate = 20000  # Hz
n_samples = len(waveform_mean)
time_axis = np.arange(n_samples) / sampling_rate * 1000  # Convert to milliseconds

# Plot waveform with error bars
plt.figure(figsize=(10, 6))
plt.plot(time_axis, waveform_mean, 'b-', linewidth=2, label='Mean waveform')
plt.fill_between(time_axis, 
                    waveform_mean - waveform_sd, 
                    waveform_mean + waveform_sd, 
                    alpha=0.3, color='blue', label='±1 SD')

plt.xlabel('Time (ms)')
plt.ylabel('Voltage (μV)')
plt.title(f'Unit {unit_id} - Mean Spike Waveform')
plt.legend()
plt.grid(True, alpha=0.3)

# Add waveform characteristics
duration = nwbfile.units['waveform_duration_ms'][unit_id]
cell_type = nwbfile.units['cell_type'][unit_id]

plt.text(0.7, 0.95, f'Duration: {duration:.2f} ms', 
        transform=plt.gca().transAxes, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
plt.text(0.7, 0.85, f'Cell type: {cell_type}', 
        transform=plt.gca().transAxes, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

## Antidromic Stimulation Analysis

Examine the antidromic stimulation data used for cell type identification.

In [ ]:
# Analyze antidromic stimulation data
antidromic_module = nwbfile.processing['antidromic_identification']
print(f"Antidromic identification module found")
print(f"Description: {antidromic_module.description}")

# Find stimulation and response series
stim_series = []
response_series = []

for name, obj in antidromic_module.data_interfaces.items():
    if 'Stimulation' in name:
        stim_series.append((name, obj))
    elif 'Response' in name:
        response_series.append((name, obj))

print(f"\\nFound {len(stim_series)} stimulation series and {len(response_series)} response series")

# Plot one stimulation-response pair
stim_name, stim_data = stim_series[0]
resp_name, resp_data = response_series[0]

print(f"\\nAnalyzing: {stim_name} and {resp_name}")

# Get a small segment for visualization (first 5 sweeps)
n_samples_per_sweep = 1000  # 50ms at 20kHz
n_sweeps_to_plot = 5

stim_segment = stim_data.data[:n_samples_per_sweep * n_sweeps_to_plot]
resp_segment = resp_data.data[:n_samples_per_sweep * n_sweeps_to_plot]
time_segment = stim_data.timestamps[:n_samples_per_sweep * n_sweeps_to_plot]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot stimulation current
ax1.plot(time_segment, stim_segment * 1e6, 'r-', linewidth=1)  # Convert to μA
ax1.set_ylabel('Stimulation Current (μA)')
ax1.set_title(f'{stim_name} - First {n_sweeps_to_plot} sweeps')
ax1.grid(True, alpha=0.3)

# Plot neural response
ax2.plot(time_segment, resp_segment * 1e6, 'b-', linewidth=1)  # Convert to μV
ax2.set_ylabel('Neural Response (μV)')
ax2.set_xlabel('Time (s)')
ax2.set_title(f'{resp_name} - First {n_sweeps_to_plot} sweeps')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Stimulation data shape: {stim_data.data.shape}")
print(f"Response data shape: {resp_data.data.shape}")
print(f"Stimulation placed at: {stim_data.timestamps[0]:.1f} seconds after session start")

## Cross-Data Analysis: Spike-Triggered Averages

Demonstrate how to combine multiple data streams for analysis.

In [ ]:
# Create spike-triggered average of kinematic data
unit_id = 0
spike_times = nwbfile.units['spike_times'][unit_id]

# Get kinematic data
kinematic_series = nwbfile.acquisition['TimeSeriesElbowVelocity']

# Parameters for spike-triggered average
window_size = 0.5  # ±500ms around each spike
sampling_rate = kinematic_series.rate
window_samples = int(window_size * sampling_rate)

# Get kinematic data
kinematic_data = kinematic_series.data[:]

# Extract windows around spikes
sta_windows = []

for spike_time in spike_times:
    spike_sample = int(spike_time * sampling_rate)
    
    # Check if window fits within data
    if (spike_sample - window_samples >= 0 and 
        spike_sample + window_samples < len(kinematic_data)):
        
        window = kinematic_data[spike_sample - window_samples:spike_sample + window_samples + 1]
        sta_windows.append(window)

sta_windows = np.array(sta_windows)

# Calculate mean and standard error
sta_mean = np.mean(sta_windows, axis=0)
sta_sem = np.std(sta_windows, axis=0) / np.sqrt(len(sta_windows))

# Create time axis
time_axis = np.linspace(-window_size, window_size, len(sta_mean))

# Plot spike-triggered average
plt.figure(figsize=(10, 6))
plt.plot(time_axis, sta_mean, 'b-', linewidth=2, label='Mean')
plt.fill_between(time_axis, 
               sta_mean - sta_sem, 
               sta_mean + sta_sem, 
               alpha=0.3, color='blue', label='±SEM')

plt.axvline(0, color='red', linestyle='--', alpha=0.7, label='Spike time')
plt.xlabel('Time relative to spike (s)')
plt.ylabel('Elbow Velocity')
plt.title(f'Spike-Triggered Average (Unit {unit_id}, n={len(sta_windows)} spikes)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Analyzed {len(sta_windows)} spikes out of {len(spike_times)} total")

## Summary

This notebook demonstrates the key data types and analysis approaches for the Turner Lab M1 MPTP dataset:

**Data Types Covered:**
1. **Electrode configuration**: Recording and stimulation electrode setup with anatomical coordinates
2. **Single-unit activity**: Spike times, waveforms, and cell type classification
3. **Motor behavior**: Trial structure and analog kinematic recordings
4. **Antidromic stimulation**: Electrical stimulation protocols for cell type identification
5. **Cross-modal analysis**: Combining spike times with kinematic data

**Key Features:**
- All temporal data maintains original accuracy within each session
- Systematic electrode mapping with chamber-relative coordinates
- Cell type identification through antidromic stimulation
- Motor task data for studying parkinsonian deficits
- Rich metadata for experimental context

**Temporal Limitations (Important):**
- Session start times are set to midnight with systematic offsets for same-day recordings
- Inter-trial intervals use fixed 3-second separation (not original behavioral timing)
- All relative timing within sessions maintains original accuracy

This standardized NWB format enables reproducible analysis of motor cortex function in parkinsonian primates and facilitates comparison with other neurophysiology datasets.